In [2]:
    # import necessary libraries
    import subprocess
    import os
    import pandas as pd
    import matplotlib.pyplot as plt
    import itertools
    import numpy as np
    import seaborn as sns
    import glob
    import cv2

    from sklearn import datasets, linear_model
    from sklearn.metrics import mean_squared_error, r2_score
    from sklearn import preprocessing
    from sklearn.metrics import accuracy_score
    from scipy import stats
    from scipy.stats import norm
    from PIL import ImageFile, Image
    from matplotlib import image,pyplot
    from PIL import ImageFile, Image
    from sklearn.model_selection import train_test_split
from mpl_toolkits import mplot3d
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [3]:
# training,test,devel data index extraction from partition file
partition  = pd.read_csv(f"/Thesis_data/c3_muse_stress_2022/metadata/partition.csv")
partition_train_index=partition[(partition["Partition"]=="train")]["Id"].to_numpy()
partition_devel_index=partition[(partition["Partition"]=="devel")]["Id"].to_numpy()
partition_test_index=partition[(partition["Partition"]=="test")]["Id"].to_numpy()
partition_index = partition[(partition["Partition"]=="train") | (partition["Partition"]=="devel")|(partition["Partition"]=="test")]["Id"].to_numpy()

In [5]:
model_val=torch.load('D:/Thesis_code/model_muse/stress/2023-03-02-17-45_[early-fusion-faus+egemaps+biosignals_noise]_[valence]_[64_4_True_64]_[0.001_256]_GRU/model_109.pth')

In [9]:
model_val.eval()

Model(
  (inp): Linear(in_features=111, out_features=64, bias=False)
  (rnn): RNN(
    (rnn): LSTM(64, 64, num_layers=4, dropout=0.2, bidirectional=True)
  )
  (out): OutLayer(
    (fc_1): Sequential(
      (0): Linear(in_features=128, out_features=64, bias=True)
      (1): ReLU(inplace=True)
      (2): Dropout(p=0.5, inplace=False)
    )
    (fc_2): Linear(in_features=64, out_features=1, bias=True)
  )
  (final_activation): Tanh()
)

In [142]:
from sklearn.preprocessing import StandardScaler
multimodal=[]
for i in partition_train_index:
    multimodal.append(pd.read_csv(f"/Thesis_data/c3_muse_stress_2022/feature_segments/biosignals/{i}.csv"))  
#     normalizer = StandardScaler().fit(multi)
pandas=pd.concat(multimodal)

scaler = StandardScaler().fit(pandas[['ECG','resp','BPM']])


X_scaled = scaler.transform(pandas)
pandas

C:\Users\Harsha\anaconda3\envs\muse2022\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- subject_id
- timestamp

  warnings.warn(message, FutureWarning)


ValueError: X has 5 features, but StandardScaler is expecting 3 features as input.

In [36]:
bio_signals_validation=pd.DataFrame()
for i in partition_devel_index:
    multi=pd.read_csv(f"/Thesis_data/c3_muse_stress_2022/feature_segments/early-fusion-faus+egemaps+biosignals/{i}.csv")
#     print(multi)
    val_value=pd.read_csv(f"/Thesis_data/results_gpu/prediction_muse/stress/2023-03-02-17-45_[early-fusion-faus+egemaps+biosignals_noise]_[valence]_[64_4_True_64]_[0.001_256]/predictions.csv")
    normalizer.transform(multi)
    multi['preds']=val_value['prediction']
    bio_signals_validation = pd.concat([bio_signals_validation,multi])
# bio_signals_validation['BPM'] = zscore(bio_signals_validation['BPM'])
bio_signals_validation

,timestamp,subject_id,AU01,AU02,AU04,AU05,AU06,AU07,AU09,AU10,...,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,equivalentSoundLevel_dBp,ECG,resp,BPM,preds
0,0,6,0.615363,0.474530,0.375809,0.202469,0.395159,0.493402,0.389130,0.767940,...,0.8,0.065,0.022913,1.165000,1.293687,-40.90551,-0.000839,4.463251,104.877610,0.057984
1,500,6,0.640970,0.460509,0.384374,0.175210,0.317188,0.466433,0.359633,0.644639,...,1.0,0.128,0.142744,0.852000,1.038641,-40.02984,-0.003710,4.974696,107.317820,0.076819
2,1000,6,0.606938,0.469241,0.380405,0.183375,0.181719,0.361823,0.284399,0.290171,...,1.0,0.212,0.183891,0.638333,0.813786,-39.42151,-0.000016,3.749285,108.521425,0.093067
3,1500,6,0.572770,0.407671,0.414971,0.235690,0.131203,0.315341,0.178140,0.189632,...,1.0,0.212,0.183891,0.638333,0.603253,-38.81771,-0.006471,3.032499,108.715908,0.107203
4,2000,6,0.578780,0.446455,0.431642,0.180185,0.204954,0.412987,0.318393,0.385922,...,1.0,0.212,0.183891,0.638333,0.481920,-38.24644,-0.016000,2.634568,108.743907,0.119334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,296500,70,0.612330,0.475978,0.359864,0.128778,0.134671,0.279034,0.274255,0.154390,...,0.0,0.000,0.000000,4.990000,0.000000,-61.93516,-0.008892,-0.851584,74.349442,0.128197
594,297000,70,0.560818,0.417507,0.246447,0.320851,0.038320,0.118362,0.033326,0.148044,...,0.0,0.000,0.000000,4.990000,0.000000,-61.93516,0.010257,-0.904601,73.000904,0.131263
595,297500,70,0.504809,0.341356,0.228822,0.321594,0.062533,0.113262,0.049096,0.136081,...,0.0,0.000,0.000000,4.990000,0.000000,-61.93516,0.006972,-0.884108,72.457989,0.133895
596,298000,70,0.537397,0.403090,0.244636,0.368199,0.051263,0.122087,0.020647,0.147996,...,0.0,0.000,0.000000,4.990000,0.000000,-61.93516,-0.027886,-0.684664,70.093458,0.136347


In [93]:
# col = list(data.columns)
for i in partition.Id:
#     if i in partition_devel_index:
    multi=pd.read_csv(f"/Thesis_data/c3_muse_stress_2022/feature_segments/early-fusion-faus+egemaps+biosignals/{i}.csv")
#         col.remove('resp')
#         col.remove('subject_id')
#         col.remove('timestamp')
    multi[col] = 0
    multi.to_csv(f"D:/Thesis_data/c3_muse_stress_2022/feature_segments/early-fusion-faus+egemaps+biosignals_colearning/{i}.csv",index=False)

           

In [175]:
data = pd.read_csv(f"/Thesis_data/c3_muse_stress_2022/feature_segments/early-fusion-egemaps+biosignals/6.csv")

In [176]:
col = list(data.columns)
# col['F0semitoneFrom27.5Hz_sma3nz_amean']

In [108]:
# col.remove('ECG')
# col.remove('BPM')
# col.remove('resp')
# col.remove('subject_id')
# col.remove('timestamp')

In [185]:
# data[col] = 0
data[col[:-3]]


,timestamp,subject_id,F0semitoneFrom27.5Hz_sma3nz_amean,F0semitoneFrom27.5Hz_sma3nz_stddevNorm,F0semitoneFrom27.5Hz_sma3nz_percentile20.0,F0semitoneFrom27.5Hz_sma3nz_percentile50.0,F0semitoneFrom27.5Hz_sma3nz_percentile80.0,F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2,F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope,...,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,spectralFluxUV_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,equivalentSoundLevel_dBp
0,500,6,24.08997,0.054085,23.26606,23.31490,26.13357,2.867514,94.83669,9.797161,...,0.051662,0.004130,0.065951,1.4,0.8,0.065000,0.022913,1.165000,1.293687,-40.90551
1,1000,6,25.04984,0.062938,23.27943,26.17496,26.36228,3.082853,94.83669,9.797161,...,0.052437,0.002865,0.065979,1.4,1.0,0.128000,0.142744,0.852000,1.038641,-40.02984
2,1500,6,24.18889,0.074080,22.30092,23.46593,26.31814,4.017216,67.17855,39.924110,...,0.052905,0.001568,0.066078,1.8,1.0,0.212000,0.183891,0.638333,0.813786,-39.42151
3,2000,6,24.18889,0.074080,22.30092,23.46593,26.31814,4.017216,67.17855,39.924110,...,0.055410,-0.000949,0.071730,2.4,1.0,0.212000,0.183891,0.638333,0.603253,-38.81771
4,2500,6,24.18889,0.074080,22.30092,23.46593,26.31814,4.017216,67.17855,39.924110,...,0.057245,-0.003954,0.077342,2.6,1.0,0.212000,0.183891,0.638333,0.481920,-38.24644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597,299000,6,24.00682,0.157237,22.39437,22.76002,24.76847,2.374092,270.14400,437.870200,...,0.071842,-0.008535,0.082836,3.2,2.6,0.135385,0.099120,0.227692,0.225769,-35.21375
598,299500,6,24.00682,0.157237,22.39437,22.76002,24.76847,2.374092,270.14400,437.870200,...,0.071842,-0.008535,0.082836,3.2,2.6,0.135385,0.099120,0.227692,0.225769,-35.21375
599,300000,6,24.00682,0.157237,22.39437,22.76002,24.76847,2.374092,270.14400,437.870200,...,0.071842,-0.008535,0.082836,3.2,2.6,0.135385,0.099120,0.227692,0.225769,-35.21375
600,300500,6,24.00682,0.157237,22.39437,22.76002,24.76847,2.374092,270.14400,437.870200,...,0.071842,-0.008535,0.082836,3.2,2.6,0.135385,0.099120,0.227692,0.225769,-35.21375


In [88]:
partition.Id

0      1
1      2
2      3
3      4
4      5
      ..
64    69
65    70
66    71
67    72
68    73
Name: Id, Length: 69, dtype: int64

In [186]:
# col = list(data.columns)
for i in partition.Id:
#     if i in partition_devel_index:
    multi=pd.read_csv(f"/Thesis_data/c3_muse_stress_2022/feature_segments/early-fusion-egemaps+biosignals/{i}.csv")
#         col.remove('resp')
#         col.remove('subject_id')
#         col.remove('timestamp')
    multi[col[:-3]] = 0
    multi.to_csv(f"D:/Thesis_data/c3_muse_stress_2022/feature_segments/early-fusion-egemaps+biosignals_colearning/{i}.csv",index=False)

           

In [136]:
from sklearn.preprocessing import StandardScaler
bio=[]
for i in partition_train_index:
    bio.append(pd.read_csv(f"/Thesis_data/c3_muse_stress_2022/feature_segments/biosignals/{i}.csv"))  
#     normalizer = StandardScaler().fit(multi)
pandas=pd.concat(bio)

scaler = StandardScaler().fit(pandas)


X_scaled = scaler.transform(pandas)
np.shape(X_scaled)


(24712, 5)

In [167]:
# col = list(data.columns)
for i in partition.Id:
#     if i in partition_devel_index:
    multi=pd.read_csv(f"/Thesis_data/c3_muse_stress_2022/feature_segments/biosignals/{i}.csv")
#         col.remove('resp')
#         col.remove('subject_id')
#         col.remove('timestamp')
    multi[col[2:4]] = 0
#     multi['BPM'] = zscore(multi['BPM'])
    multi.to_csv(f"D:/Thesis_data/c3_muse_stress_2022/feature_segments/biosignals_colearning/{i}.csv",index=False)

           

In [203]:
import pandas as pd

partition  = pd.read_csv(f"/media/compute/homes/hmanjunath/c3_muse_stress_2022/metadata/partition.csv")

for i in partition.Id:
    combine=pd.DataFrame()

    valence=pd.read_csv(f"/media/compute/homes/hmanjunath/c3_muse_stress_2022/label_segments/valence/{i}.csv")
    arousal=pd.read_csv(f"/media/compute/homes/hmanjunath/c3_muse_stress_2022/label_segments/physio-arousal/{i}.csv")
    combine[['timestamp','subject_id','v_value']]=valence[['timestamp','subject_id','value']]
    combine['a_value']=arousal['value']
    combine.to_csv(f"/media/compute/homes/hmanjunath/c3_muse_stress_2022/label_segments/valence_arousal1/{i}.csv",index=False)

In [ ]:
import pandas as pd

# partition  = pd.read_csv(f"/media/compute/homes/hmanjunath/c3_muse_stress_2022/metadata/partition.csv")

for i in partition_index:
    combine=pd.DataFrame()
    print(i)
    valence=pd.read_csv(f"/Thesis_data/c3_muse_stress_2022/label_segments/valence/{i}.csv")
    arousal=pd.read_csv(f"/Thesis_data/c3_muse_stress_2022/label_segments/physio-arousal/{i}.csv")
    combine[['timestamp','subject_id','v_value']]=valence[['timestamp','subject_id','value']]
    combine['a_value']=arousal['value']
    combine.to_csv(f"/Thesis_data/c3_muse_stress_2022/label_segments/valence_arousal/{i}.csv",index=False)